# SQL Codegen SLM - Training Notebook

Fine-tune Mistral-7B for PostgreSQL query generation using LoRA and 4-bit quantization.

**Requirements:**
- Google Colab Pro+ (for A100 GPU access)
- GCP Project with Cloud Storage bucket
- ~8-12 hours training time

**Storage:** Google Cloud Storage (faster than Drive)

## 1. Check GPU Allocation

First, verify you have a GPU. Ideally A100 (40GB), V100 (16GB) is okay, T4 will be slower.

In [ ]:
!nvidia-smi

import torch
if torch.cuda.is_available():
    gpu_name = torch.cuda.get_device_name(0)
    gpu_mem = torch.cuda.get_device_properties(0).total_memory / 1e9
    print(f"\n✅ GPU: {gpu_name} ({gpu_mem:.1f} GB)")
    
    if "A100" in gpu_name:
        print("🎉 Got A100 - optimal for training!")
    elif "V100" in gpu_name:
        print("⚠️ Got V100 - good, but A100 is faster")
    elif "T4" in gpu_name:
        print("⚠️ Got T4 - training will be slower (~2x)")
        print("💡 Tip: Disconnect and reconnect to try for A100")
else:
    print("❌ No GPU! Go to Runtime > Change runtime type > GPU")

## 2. Configure GCP Project

Set your GCP project ID and bucket name.

In [ ]:
# ⚠️ UPDATE THESE VALUES
PROJECT_ID = "your-gcp-project-id"  # Your GCP project ID
BUCKET_NAME = "sql-codegen-slm-data"  # Your GCS bucket name

# Set environment variables
import os
os.environ["GCP_PROJECT_ID"] = PROJECT_ID
os.environ["GCS_BUCKET"] = BUCKET_NAME

print(f"Project: {PROJECT_ID}")
print(f"Bucket: gs://{BUCKET_NAME}")

## 3. Authenticate with Google Cloud

This will open a popup to authenticate with your Google account.

In [ ]:
from google.colab import auth
auth.authenticate_user()

# Configure gcloud
!gcloud config set project {PROJECT_ID}

print("\n✅ Authenticated with Google Cloud")

## 4. Clone Repository & Install Dependencies

In [ ]:
# Clone repository (update with your GitHub username)
!git clone https://github.com/YOUR_USERNAME/sql-codegen-slm.git
%cd sql-codegen-slm

# Install dependencies
!pip install -q -r training/requirements.txt

print("\n✅ Dependencies installed")

## 5. Setup GCS Bucket & Download Data

This will:
1. Create the GCS bucket if it doesn't exist
2. Download training data from GCS to local storage

In [ ]:
# Create bucket if needed
!gsutil ls gs://{BUCKET_NAME} 2>/dev/null || gsutil mb -l us gs://{BUCKET_NAME}

# Create local directories
!mkdir -p /content/data /content/models /content/logs /content/tensorboard

# Download data from GCS
!gsutil -m cp gs://{BUCKET_NAME}/data/*.jsonl /content/data/ 2>/dev/null || echo "Data not in GCS yet"

# Check what we have
!ls -la /content/data/

## 5b. Upload Data to GCS (if not already there)

**Run this on your LOCAL machine first:**
```bash
# From your sql-codegen-slm directory
gsutil -m cp data/processed/*.jsonl gs://YOUR_BUCKET_NAME/data/
```

Or upload directly from Colab:

In [ ]:
# Option: Upload files directly to Colab, then to GCS
# Uncomment if you need to upload data

# from google.colab import files
# uploaded = files.upload()  # Select your JSONL files
# !mv *.jsonl /content/data/
# !gsutil -m cp /content/data/*.jsonl gs://{BUCKET_NAME}/data/

## 6. Verify Environment

In [ ]:
from training.colab_setup import setup_colab_environment, estimate_training_time

# Full environment check
status = setup_colab_environment(PROJECT_ID, BUCKET_NAME)

# Estimate training time
print("\n")
estimate_training_time()

## 7. Start Training

This will:
- Load Mistral-7B with 4-bit quantization
- Apply LoRA adapters
- Train for 3 epochs
- Save checkpoints every 500 steps

**Estimated time:** 8-12 hours on A100

In [ ]:
# Start training
!python -m training.train --config training/configs/mistral_lora_config.yaml

# If training was interrupted, resume from checkpoint:
# !python -m training.train --config training/configs/mistral_lora_config.yaml --resume

## 8. Sync Checkpoints to GCS

Run this periodically to backup checkpoints to GCS.

In [ ]:
# Sync checkpoints to GCS
!gsutil -m rsync -r /content/models gs://{BUCKET_NAME}/models/

print(f"\n✅ Checkpoints synced to gs://{BUCKET_NAME}/models/")

## 9. Monitor Training with TensorBoard

In [ ]:
%load_ext tensorboard
%tensorboard --logdir /content/tensorboard

## 10. Test the Model

After training completes, test the model with sample queries.

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

model_path = "/content/models"

tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    torch_dtype=torch.float16,
    device_map="auto",
)

print("✅ Model loaded")

In [ ]:
# Test with a sample query
schema = """
CREATE TABLE customers (
    id SERIAL PRIMARY KEY,
    name VARCHAR(100),
    email VARCHAR(100)
);

CREATE TABLE orders (
    id SERIAL PRIMARY KEY,
    customer_id INTEGER REFERENCES customers(id),
    total DECIMAL(10,2),
    created_at TIMESTAMP
);
"""

question = "Find customers who have placed more than 5 orders"

prompt = f"""[INST] Given the following PostgreSQL schema:

{schema}

Write a SQL query to: {question} [/INST]"""

inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=256,
        temperature=0.1,
        do_sample=True,
    )

response = tokenizer.decode(outputs[0], skip_special_tokens=True)
sql = response.split("[/INST]")[-1].strip()

print("📝 Question:", question)
print("\n🔍 Generated SQL:")
print(sql)

## 11. Final Sync to GCS

Upload final model and logs to GCS for permanent storage.

In [ ]:
# Sync everything to GCS
!gsutil -m rsync -r /content/models gs://{BUCKET_NAME}/models/
!gsutil -m rsync -r /content/logs gs://{BUCKET_NAME}/logs/
!gsutil -m rsync -r /content/tensorboard gs://{BUCKET_NAME}/tensorboard/

print(f"\n✅ All files synced to gs://{BUCKET_NAME}/")
print(f"\nView in console: https://console.cloud.google.com/storage/browser/{BUCKET_NAME}")

---

## Troubleshooting

### Session Disconnected?
1. Reconnect to Colab
2. Run cells 1-6 again (auth, setup)
3. Download latest checkpoint: `!gsutil -m cp -r gs://{BUCKET_NAME}/models/* /content/models/`
4. Resume training with `--resume` flag

### Out of Memory?
- Reduce batch size: `per_device_train_batch_size: 2`
- Increase gradient accumulation: `gradient_accumulation_steps: 8`

### GCS Permission Denied?
- Re-run authentication cell
- Check bucket permissions in GCP Console